### Interactive Choropleth Maps

In [1]:
from pathlib import Path

import geopandas as gpd
import pandas as pd

import shapely
import plotly.express as px

from helper_functions import show_overlaps, boundary_cut, plot_bounds

In [2]:
file_path = Path("data/Municipal_Boundaries_SHP_Geographic/")

In [3]:
rural_gdf = gpd.read_file(file_path/"RURAL.shp") #1 
reserve_gdf = gpd.read_file(file_path/"INDIAN.shp") # 2
city_gdf = gpd.read_file(file_path/"CITY.shp") # 3
town_gdf = gpd.read_file(file_path/"TOWN.shp") # 4 
urbserv_gdf = gpd.read_file(file_path/"urbserv.shp") # 5
# hamlet_df = gpd.read_file(file_path/"HAMLETPT.shp") # points, no boundaries
village_gdf = gpd.read_file(file_path/"VILLAGE.shp") # 6
svillage_gdf = gpd.read_file(file_path/"SVILLAGE.shp") # 7 

In [4]:
ookla_gdf = gpd.read_file("data/ookla/AB_ookla_data_2020.shp")

In [5]:
ookla_gdf["long"] = ookla_gdf.geometry.apply(lambda x:x.centroid.x)
ookla_gdf["lat"] = ookla_gdf.geometry.apply(lambda x:x.centroid.y)

In [6]:
ookla_temp = pd.DataFrame(ookla_gdf.drop("geometry", axis=1))
ookla_gdf = gpd.GeoDataFrame(ookla_temp, geometry=gpd.points_from_xy(
    ookla_temp.long, ookla_temp.lat))
ookla_gdf = ookla_gdf.drop(["long", "lat"], axis=1)

In [7]:
ookla_gdf.crs = {"init":"EPSG:4326"}

/home/peter/anaconda3/envs/basic-data/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [8]:
rural_cuts = [reserve_gdf, city_gdf, town_gdf, urbserv_gdf, village_gdf, svillage_gdf] 

In [9]:
rural_gdf = boundary_cut(rural_cuts, rural_gdf)

In [10]:
reserve_cuts = rural_cuts.copy()
reserve_cuts.remove(reserve_gdf)

In [11]:
reserve_gdf = boundary_cut(reserve_cuts, reserve_gdf)

In [12]:
rural_data = gpd.sjoin(ookla_gdf, rural_gdf.to_crs("+init=epsg:4326 +type=crs"))

/home/peter/anaconda3/envs/basic-data/lib/python3.8/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)


In [13]:
reserve_data = gpd.sjoin(ookla_gdf, reserve_gdf.to_crs("+init=epsg:4326 +type=crs"))

/home/peter/anaconda3/envs/basic-data/lib/python3.8/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)


In [14]:
rural_dict = rural_data.groupby("GEONAME")["avg_d_kbps"].mean().reset_index()
reserve_dict = reserve_data.groupby("GEONAME")["avg_d_kbps"].mean()

In [15]:
rural_final = rural_gdf[["GEONAME", "geometry"]].set_index("GEONAME")
reserve_final1 = reserve_gdf[["GEONAME", "geometry"]].set_index("GEONAME")

In [16]:
rural_gdf.head(3)

,RURAL_ID,PID,GEONAME,GEOCODE,geometry
0,1,29860,MUNICIPAL DISTRICT OF ACADIA NO. 34,0001,"POLYGON ((-110.05165 51.27041, -110.05136 51.2..."
1,2,29863,ATHABASCA COUNTY,0012,"POLYGON ((-113.68568 54.53633, -113.68569 54.5..."
2,3,29864,COUNTY OF BARRHEAD NO. 11,0015,"POLYGON ((-114.05164 53.92673, -114.05164 53.9..."


In [17]:
rural_final = rural_gdf.merge(rural_dict, how="inner").set_index("GEONAME")

In [18]:
# df = px.data.election()
# geo_df = gpd.GeoDataFrame.from_features(
#     px.data.election_geojson()["features"]
# ).merge(df, on="district").set_index("district")

In [20]:
rural_final.head(3)

,RURAL_ID,PID,GEOCODE,geometry,avg_d_kbps
GEONAME,,,,,
MUNICIPAL DISTRICT OF ACADIA NO. 34,1,29860,0001,"POLYGON ((-110.05165 51.27041, -110.05136 51.2...",14158.789474
ATHABASCA COUNTY,2,29863,0012,"POLYGON ((-113.68568 54.53633, -113.68569 54.5...",20790.161443
COUNTY OF BARRHEAD NO. 11,3,29864,0015,"POLYGON ((-114.05164 53.92673, -114.05164 53.9...",40068.579741


In [21]:
geo_df.head(3)

,geometry,Coderre,Bergeron,Joly,total,winner,result,district_id
district,,,,,,,,
11-Sault-au-Récollet,"MULTIPOLYGON (((-73.63632 45.57592, -73.63628 ...",3348,2770,2532,8650,Coderre,plurality,11
12-Saint-Sulpice,"POLYGON ((-73.62175 45.55448, -73.62350 45.553...",3252,2521,2543,8316,Coderre,plurality,12
13-Ahuntsic,"POLYGON ((-73.65132 45.55457, -73.65687 45.545...",2979,3430,2873,9282,Bergeron,plurality,13


In [62]:
len(geo_df.loc["12-Saint-Sulpice"].geometry.exterior.coords.xy[0])

34

In [30]:
# fig = px.choropleth(rural_final,
#                    geojson=rural_final.geometry,
#                    locations=rural_final.index,
#                    color="avg_d_kbps",
#                    projection="mercator")
# fig.update_geos(fitbounds="locations", visible=False)
# fig.show()